In [ ]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.interpolate import interpolate_to_isosurface
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from siphon.ncss import NCSS
import xarray as xr

In [ ]:
def plot_PV(level, dist, smooth=10):
    print(f"Creating the {level}-hPa PV Map")
    #ilev = list(lev.m).index(level*100.)
    subset = dict(vertical = level*units.hPa)
    uwnd_ilev = uwnd.metpy.sel(subset).metpy.convert_units('kt')
    vwnd_ilev = vwnd.metpy.sel(subset).metpy.convert_units('kt')
    
    sped_ilev = mpcalc.wind_speed(uwnd_ilev, vwnd_ilev)
    
    if abs(dist) < 1:
        div_ilev = mpcalc.smooth_n_point(div.metpy.sel(subset), 9, smooth)
        epv_smooth = mpcalc.smooth_n_point(epv.metpy.sel(subset), 9, smooth)
    else:
        div_ilev = mpcalc.zoom_xarray(div.metpy.sel(subset), 4)
        epv_smooth = mpcalc.zoom_xarray(mpcalc.smooth_n_point(epv.metpy.sel(subset), 9, smooth), 4)

    fig = plt.figure(1, figsize=(17,15))

    # 1st panel
    ax = plt.subplot(111, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cf = ax.contourf(clons, clats, sped_ilev, range(10, 230, 20), cmap=plt.cm.BuPu, extend='max')
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    cs = ax.contour(clons, clats, epv_smooth*1e6, range(2, 15, 1),
                    colors='black')
    plt.clabel(cs, fmt='%d')
    
    cs2 = ax.contour(clons, clats, div_ilev*1e5, range(1, 50, 3),
                     colors='grey', linestyles='dashed')
    plt.clabel(cs2, fmt='%d')

    plt.title(f'GFS - {level}-hPa PV (PVU), Divergence ($*10^5$ $s^{-1}$), and Wind Spped (kt)', loc='left')
    plt.title(f'Valid Time: {date}', loc='right')

    plt.savefig(f'GFS_{level}-hPa_PV_{date:%Y%m%d_%H}00.png', bbox_inches='tight', dpi=150)
    plt.show()
    #plt.close()

In [ ]:
def plot_DT(smooth=10):
    fig = plt.figure(1, figsize=(17,15))

    # 1st panel
    ax = plt.subplot(111, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cf = ax.contourf(clons, clats, mpcalc.smooth_n_point(thta_DT, 9, smooth), np.arange(258,427,6), cmap=plt.cm.coolwarm, extend='both')
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    cs = ax.contour(clons, clats,
                    mpcalc.smooth_n_point(relvor_925850, 9, smooth)*1e4,
                    np.arange(0.5,30,.5), colors='black')
    plt.clabel(cs, fmt='%d')

    cs2 = ax.contour(clons, clats, mpcalc.smooth_n_point(thta_DT, 9, smooth), np.arange(258,427,6), colors='grey', linestyles='dotted')
    #plt.clabel(cs2, fmt='%d')

    ax.barbs(lons[wind_slice], lats[wind_slice],
             uwnd_DT[wind_slice], vwnd_DT[wind_slice],
             transform=ccrs.PlateCarree())

    plt.title(r'GFS - DT Potential Temp (K), 925-850-hPa Avg. Rel. Vor. ($*10^4$ $s^{-1}$),'
              ' and Wind Spped (kt)', loc='left')
    plt.title('Valid Time: {}'.format(date), loc='right')

    plt.savefig('GFS_DT_Potential_Temp_{0:%Y%m%d_%H}00.png'.format(date), bbox_inches='tight', dpi=150)
    plt.show()
    #plt.close()

In [ ]:
date = datetime(2021, 2, 2, 12)

# Choose a dataset...
    
# Current Data
# ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p5deg_ana/TP')

# UCAR RDA Archive - Global 1.0 deg Data from August 1, 1999 to current
# if datetime(1999, 7, 30, 18) <= date <= datetime(2007, 12, 6, 0):
#     ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083002/grib1/'
#                          f'{date:%Y}/{date:%Y.%m}/fnl_{date:%Y%m%d_%H}_00.grib1')
# elif date > datetime(2007, 12, 6, 0):
#     ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083002/grib2/'
#                          f'{date:%Y}/{date:%Y.%m}/fnl_{date:%Y%m%d_%H}_00.grib2')

# UCAR RDA Archive - Global 0.25 deg Data from 2015 to current
ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083003/'
                       f'{date:%Y}/{date:%Y%m}/gdas1.fnl0p25.{date:%Y%m%d%H}.f00.grib2')

# Local File
#ds = xr.open_dataset(f'groundhogs_day_blizzard/GFS_{date:%Y%m%d}_{date:%H}00.nc')

subset = dict(vertical=slice(70 * units.hPa, 1000*units.hPa),
              time=date,
              lat=slice(60, 10),
              lon=slice(360-160, 360-50))

tmpk = ds['Temperature_isobaric'].metpy.sel(subset).squeeze()
rename_dict = {ds['u-component_of_wind_isobaric'].metpy.vertical.name: tmpk.metpy.vertical.name}
uwnd = ds['u-component_of_wind_isobaric'].rename(rename_dict).metpy.sel(subset).squeeze()
vwnd = ds['v-component_of_wind_isobaric'].rename(rename_dict).metpy.sel(subset).squeeze()

lev = tmpk.metpy.vertical.values * units(tmpk.metpy.vertical.units)

dist = tmpk.lat[1] - tmpk.lat[0]

if abs(dist) == .25:
    relvor = mpcalc.vorticity(uwnd, vwnd)
    wind_slice = (slice(None, None, 10), slice(None, None, 10))
    smooth = 20
elif abs(dist) == .5:
    relvor = mpcalc.vorticity(uwnd, vwnd)
    wind_slice = (slice(None, None, 5), slice(None, None, 5))
    smooth = 5
else:
    relvor = mpcalc.zoom_xarray(mpcalc.vorticity(uwnd, vwnd), (1, 4, 4))
    wind_slice = (slice(None, None, 2), slice(None, None, 2))
    smooth = 2  

lons, lats = np.meshgrid(relvor.lon, relvor.lat)

mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(), lons, lats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

thta = mpcalc.potential_temperature(lev[:, None, None], tmpk)

div = mpcalc.divergence(uwnd, vwnd)

epv = mpcalc.potential_vorticity_baroclinic(thta, lev[:, None, None], uwnd, vwnd)

thta_DT = interpolate_to_isosurface(epv[1:].values*1e6, thta[1:].values, 2)
uwnd_DT = interpolate_to_isosurface(epv[1:].values*1e6, uwnd[1:].metpy.convert_units('kt').values, 2)
vwnd_DT = interpolate_to_isosurface(epv[1:].values*1e6, vwnd[1:].metpy.convert_units('kt').values, 2)

ip925850 = (lev == 850*units.hPa) | (lev == 925*units.hPa)
relvor_925850 = relvor[ip925850].mean(axis=0)

In [ ]:
plot_PV(250, dist, smooth)

In [ ]:
plot_DT(smooth)